# Welcome to COVID-19 data insight


- The purpose of this site is to provide some additional/complementary visualization that are not typically shown by other covid-19 sites. 
- The data used in this site is from [NY Times COVID-19 data](https://github.com/nytimes/covid-19-data)
- Please send your comments and suggestions to: [jianganghao@gmail.com](jianganghao@gmail.com)


---

In [9]:
import pandas as pd
import plotly.express as px
import cufflinks as cf
from ipywidgets import interact, fixed, widgets, Layout
from plotly.subplots import make_subplots
from IPython.display import display
import numpy as np
cf.go_offline()
cf.getThemes()
cf.set_config_file(theme='space')

In [2]:
# data from NY times github
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
dft = df.pivot_table(values='cases', index='date',columns='state',aggfunc='sum').rolling(7).mean()

states = df.state.unique().tolist()
states.sort()


In [3]:
date_list = dft.index.tolist()
initial_date = widgets.SelectionSlider(options=[dt for dt in date_list], value='2020-06-01', \
                                       description='Starting', disabled=False, continuous_update=False\
                                       , orientation='horizontal', readout=True,layout=Layout(width='50%', height='40px'))
ending_date = widgets.SelectionSlider(options=[dt for dt in date_list], value='2020-06-15', \
                                       description='Ending', disabled=False, continuous_update=False\
                                       , orientation='horizontal', readout=True,layout=Layout(width='50%', height='40px'))

### The following plots display the average "acceleration" of the new cases among consecutive days
- Top panel shows the average acceleration for each state
- Bottom panel shows the epi-curve for states with positive acceleration (increase) and negative acceleration (decrease)

In [4]:
def acceleration_plot(ini_date,end_date,df):
    res=display(res=df.loc[(df.index>=ini_date)&(df.index<= end_date),:].diff().diff().mean()\
    .iplot(kind='bar',yTitle='Average acceleration of new cases'\
           ,title='Average acceleration between '+ini_date+' and '+end_date+' (after 7 day rolling average)',colorscale='accent',hline=0))   
    state_pos = df.loc[(df.index>=ini_date)&(dft.index <= end_date),:].diff().diff().mean()[df.loc[(df.index>=ini_date)&(df.index <= end_date),:].diff().diff().mean()>=0].index.tolist()
    state_neg = df.loc[(df.index>=ini_date)&(dft.index <= end_date),:].diff().diff().mean()[df.loc[(df.index>=ini_date)&(df.index <= end_date),:].diff().diff().mean()<0].index.tolist()
    df_pos = pd.DataFrame(df.loc[:,state_pos].sum(axis=1).diff())
    df_pos.columns = ['increase_state']
    df_neg = pd.DataFrame(df.loc[:,state_neg].sum(axis=1).diff())
    df_neg.columns = ['decrease_state']
    df_state = pd.merge(df_pos,df_neg,on='date')
    res=display(res=df_state.round().iplot(kind='bar',title='States with increase and decrease new COVID-19 cases (7 day rolling average)', yTitle='Number of new cases'))
    return 
res = interact(acceleration_plot,ini_date=initial_date,end_date=ending_date,df=fixed(dft))

interactive(children=(SelectionSlider(continuous_update=False, description='Starting', index=132, layout=Layou…

---

### The following plot shows the epi-curve and acceleration for each state

In [5]:
def epc(df,state):
    res = display(res=df.loc[:,state].iplot(kind='bar',yTitle='Number of new cases',title='epi-curve by state (7 day rolling average)'))
    res = display(res=df.loc[:,state].diff().rolling(7).mean().iplot(kind='bar',yTitle='Increase/decrease number of new cases',title='Acceleration by state (7 day rolling average)'))
    return

state_list = widgets.SelectionSlider(options=[st for st in states], value='New Jersey', \
                                       description='Choose:', disabled=False, continuous_update=False\
                                       , orientation='horizontal', readout=True,layout=Layout(width='50%', height='40px'))
res = interact(epc,df=fixed(dft.diff().round()),state=state_list)
    

interactive(children=(SelectionSlider(continuous_update=False, description='Choose:', index=31, layout=Layout(…

---

### The following is an animation of the number of cases by state

In [12]:
df_state = df.groupby(['state','date']).apply(lambda g: g.cases.sum()).reset_index()
df_state.columns=['state','date','cases']
df_state['log_cases'] = np.log(df_state.cases)
df_state.sort_values('date',inplace=True)
px.bar_polar(df_state,r='cases',theta='state',animation_frame='date',template="plotly_dark",\
             color='log_cases',log_r=True,height=700)

---